In [ ]:
import tensorflow as tf
from datetime import datetime

In [ ]:
%load_ext tensorboard

In [ ]:
class SimpleModule(tf.Module):
    def __init__(self, name=None):
        super().__init__(name=name)
        self.a_variable = tf.Variable(5.0, name='train_me')
        self.non_trainable_value = tf.Variable(5.0, trainable=False, name='not_train_me')
    
    def __call__(self, x):
        return self.a_variable * x + self.non_trainable_value

simple_module = SimpleModule(name='simple')
simple_module(tf.constant(5.0))

In [ ]:
print("Trainable variables: ", simple_module.trainable_variables)
print("All variables: ", simple_module.variables)

In [ ]:
class Dense(tf.Module):
    def __init__(self, in_features, out_features, name=None):
        super().__init__(name=name)
        self.w = tf.Variable(tf.random.normal([in_features, out_features]), name='w')
        self.b = tf.Variable(tf.zeros([out_features]), name='b')
    
    def __call__(self, x):
        y = tf.matmul(x, self.w) + self.b

        return tf.nn.relu(y)

In [ ]:
class SequentialModel(tf.Module):
    def __init__(self, name=None):
        super().__init__(name=name)

        self.dense_1 = Dense(3, 3, 'dense1')
        self.dense_2 = Dense(3, 2, 'dense2')
    
    def __call__(self, x):
        x = self.dense_1(x)
        return self.dense_2(x)

my_model = SequentialModel('the_model')

print("Model result: ", my_model(tf.constant([[1., 2., 3.]])))

In [ ]:
print("Submodels: ", my_model.submodules)

In [ ]:
for var in my_model.variables:
    print(var, '\n')

In [ ]:
class FlexibleDenseLayer(tf.Module):
    def __init__(self, out_features, name=None):
        super().__init__(name=name)

        self.is_built=False
        self.out_features = out_features

    def __call__(self, x):
        if not self.is_built:
            self.w = tf.Variable(tf.random.normal([x.shape[-1], self.out_features]), name='w')
            self.b = tf.Variable(tf.zeros([self.out_features]), name='b')
            self.is_built = True

        y = tf.matmul(x, self.w) + self.b
        return tf.nn.relu(y)  

In [ ]:
class MySequentialModule(tf.Module):
    def __init__(self, name=None):
        super().__init__(name=name)
        self.dense_1 = FlexibleDenseLayer(3)
        self.dense_2 = FlexibleDenseLayer(2)
    
    def __call__(self, x):
        x = self.dense_1(x)
        return self.dense_2(x)

In [ ]:
my_model = MySequentialModule(name='my_module')
print("Module result: ", my_model(tf.constant([[2.0, 2.0, 2.0]])))

In [ ]:
chkp_path = 'my_checkpoint'
checkpoint = tf.train.Checkpoint(model=my_model)
checkpoint.write(chkp_path)

In [ ]:
tf.train.list_variables(chkp_path)

In [ ]:
new_model = MySequentialModule()
new_checkpoint = tf.train.Checkpoint(model=new_model)
new_checkpoint.restore("my_checkpoint")

print(new_model(tf.constant([[2.0, 2.0, 2.0]])))

In [ ]:
class MySequentialModule(tf.Module):
    def __init__(self, name=None):
        super().__init__(name=name)

        self.dense_1 = Dense(in_features=3, out_features=3, name='dense1')
        self.dense_2 = Dense(in_features=3, out_features=2, name='dense2')
    
    @tf.function
    def __call__(self, x):
        x = self.dense_1(x)
        return self.dense_2(x)

my_model = MySequentialModule('graph_model')

In [ ]:
print(my_model([[2.0, 2.0, 2.0], [2.0, 2.0, 2.0]]))
print(my_model([[[2.0, 2.0, 2.0], [2.0, 2.0, 2.0]]]))

In [ ]:
stamp = datetime.now().strftime("%Y%m%d-%H%M%S")
logdir = "logs/func/%s" % stamp
writer = tf.summary.create_file_writer(logdir)

new_model = MySequentialModule()

tf.summary.trace_on(graph=True)
tf.profiler.experimental.start(logdir=logdir)
z = print(new_model(tf.constant([[2.0, 2.0, 2.0]])))
with writer.as_default():
    tf.summary.trace_export(name='my_func_trace', step=0, profiler_outdir=logdir)

In [ ]:
tf.saved_model.save(my_model, 'the_saved_model')

In [ ]:
!ls -l the_saved_model

In [ ]:
!ls -l the_saved_model/variables

In [ ]:
new_model = tf.saved_model.load('the_saved_model')

In [ ]:
isinstance(new_model, SequentialModel)

In [ ]:
print(new_model([[2.0, 2.0, 2.0], [2.0, 2.0, 2.0]]))
print(new_model([[[2.0, 2.0, 2.0], [2.0, 2.0, 2.0]]]))

In [ ]:
class MyDense(tf.keras.layers.Layer):
    def __init__(self, in_features, out_features, **kwargs):
        super().__init__(**kwargs)

        self.w = tf.Variable(tf.random.normal([in_features, out_features]), name='w')
        self.b = tf.Variable(tf.zeros([out_features]), name='b')
    
    def call(self, x):
        y = tf.matmul(x, self.w) + self.b
        return tf.nn.relu(y)

simple_layer = MyDense(name='simple', in_features=3, out_features=3)

In [ ]:
simple_layer([[2.0, 2.0, 2.0]])

In [ ]:
class FlexibleDense(tf.keras.layers.Layer):
    def __init__(self, out_features, **kwargs):
        super().__init__(**kwargs)
        self.out_features = out_features
    
    def build(self, input_shape):
        self.w = tf.Variable(tf.random.normal([input_shape[-1], self.out_features]), name='w')
        self.b = tf.Variable(tf.zeros([self.out_features]), name='b')

    def call(self, x):
        y = tf.matmul(x, self.w) + self.b
        return tf.nn.relu(y)

flexible_dense = FlexibleDense(out_features=3)

In [ ]:
flexible_dense.variables

In [ ]:
print("Module result: ", flexible_dense(tf.constant([[2.0, 2.0, 2.0], [3.0, 3.0, 3.0]])))

In [ ]:
flexible_dense.variables

In [ ]:
try:
    print("Model result: ", flexible_dense(tf.constant([[4.0, 4.0, 4.0, 4.0]])))
except tf.errors.InvalidArgumentError as e:
    print('Failed: ', e)

In [ ]:
class MySequentialModel(tf.keras.Model):
    def __init__(self, name=None, **kwargs):
        super().__init__(**kwargs)
        self.dense_1 = FlexibleDense(out_features=3)
        self.dense_2 = FlexibleDense(out_features=2)
    
    def call(self, x):
        x = self.dense_1(x)
        return self.dense_2(x)

my_sequential_model = MySequentialModel(name='the_model')

In [ ]:
print("Model result: ", my_sequential_model(tf.constant([[2.0, 2.0, 2.0]])))

In [ ]:
my_sequential_model.variables

In [ ]:
my_sequential_model.submodules

In [ ]:
inputs = tf.keras.Input(shape=[3, ])

x = FlexibleDense(3)(inputs)
x = FlexibleDense(2)(x)

my_functional_model = tf.keras.Model(inputs=inputs, outputs=x)

my_functional_model.summary()

In [ ]:
my_functional_model(tf.constant([[2.0, 2.0, 2.0]]))

In [ ]:
my_sequential_model.save('exname_of_file')

In [ ]:
reconstructed_model = tf.keras.models.load_model('exname_of_file')

In [ ]:
reconstructed_model(tf.constant([[2.0, 2.0, 2.0]]))